In [5]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
SELECT * FROM player_seasons;

-- create 1 row per player and an array of seasons

-- create struct
CREATE TYPE season_stats AS (
         season INTEGER,
         gp INTEGER,
         pts REAL,
         reb REAL,
         ast REAL       
      )
      
CREATE TYPE scoring_class AS ENUM('star', 'good',  'average', 'bad')

--create table     
CREATE TABLE players (
        player_name TEXT,
        height TEXT,
        college TEXT,
        country TEXT,
        draft_year TEXT,
        draft_round TEXT,
        draft_number TEXT,
        season_stats season_stats [],
        scoring_class scoring_class,
        years_since_last_season INTEGER,
        current_season INTEGER,
        PRIMARY_KEY(player_name, current_season)
        )
     
-- SELECT MIN(season) FROM player_seasons;

 -- create seed query for cumulation
INSERT INTO players
WITH yesterday AS(
        SELECT * FROM players
        WHERE current_season = 1995
),
        today AS(
        SELECT * FROM players
        WHERE current_season = 1996
)

-- insert today as 1997, 1998, etc... to mimic pipeline


SELECT
        COALESCE(t.player_name, y.player_name) AS player_name,
        COALESCE(t.height, y.height) AS height,
        COALESCE(t.college, y.college) AS college,
        COALESCE(t.country, y.country) AS country,
        COALESCE(t.draft_year, y.draft_year) AS draft_year,
        COALESCE(t.draft_round, y.draft_round) AS draft_round,
        COALESCE(t.draft_number, y.draft_number) AS draft_number,
        
        CASE WHEN y.season_stats IS NULL
                THEN ARRAY[ROW(
                        t.season, 
                        t.gp,
                        t.pts, 
                        t.reb,
                        t.ast
                        ):: season_stats]
       WHEN t.season IS NOT NULL 
                THEN y.season_stats || ARRAY[ROW(
                        t.season, 
                        t.gp,
                        t.pts, 
                        t.reb,
                        t.ast
                        ):: season_stats]
       ELSE y.season_stats
       END AS season_stats,
       CASE 
           WHEN t.season IS NOT NULL THEN
                CASE WHEN t.pts > 20 THEN 'star'
                        WHEN t.pts > 15 THEN 'good'
                        WHEN t.pts > 10 THEN 'average'
                        ELSE 'bad'
                END::scoring_class,

       CASE 
           WHEN t.season IS NOT NULL THEN 1
       END,
       
       CASE 
           WHEN t.season IS NOT NULL THEN 0
           ELSE y.years_since_last_season + 1
       END AS years_since_last_season,
      
       COALESCE(t.season, y.current__season + 1) AS current_season                                                  
FROM today t
FULL OUTER JOIN yesterday y
        ON t.player_name = y.player_name;

-- see what happens with player who retires then returns
SELECT * 
FROM players 
WHERE current_season = 2001
AND player_name = 'Michael Jordan';


-- convert back (unnest) to separate columns
WITH unnested AS (
        SELECT player_name,
                UNNEST(season_stats)::season_stats AS season_stats 
        FROM players 
        WHERE current_season = 2001
        AND player_name = 'Michael Jordan'
)

SELECT player_name,
        (season_stats::season_stats).*
FROM unnested

-- Query Analysis compare first and last seasons - does not use GROUP BY - and is therefore fast
SELECT player_name,
       (season_stats[CARDINALITY(season_stats)]::season_stats).pts / 
       CASE WHEN (season_stats[1]::season_stats).pts = 0 THEN 1, ELSE (season_stats[1]::season_stats).pts)        
FROM players
WHERE current_season = 2001
ORDER BY 2 DESC;

/* theory behind the query above
SELECT player_name,
        (season_stats[1]::season_stats).pts AS first_season,
        (season_stats[CARDINALITY(season_stats)]::season_stats).pts AS latest_season
FROM players

CASE WHEN is necessary to prevent dividing by 0

ORDER BY returns the most improved player first
*/



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 66)